In [11]:
# необходимо запустить код этой ячейки для начала работы с программой.
# программа работает после предварительной загрузки файла games.csv - выгрзука из полученного xlsx-файла

import pandas as pd
import numpy as np
import datetime
import plotly
import plotly.graph_objs as go
import plotly.express as px

def preparations():
  print('Loading data... (about 30 seconds)')
  df = pd.read_csv('games.csv', sep = '\t')
  df.columns = ['team', 'action', 'x', 'y']
  pt_total = df.groupby(['team', 'x', 'y'])['action'].agg('count').reset_index()
  pt_goals = df[df.action == 'Гол'].groupby(['team', 'x', 'y'])['action'].agg('count').reset_index()
  pt = pt_total.merge(pt_goals.action, how = 'left', left_on = [pt_total.team, pt_total.x, pt_total.y], right_on = [pt_goals.team, pt_goals.x, pt_goals.y]).fillna(0).drop(columns = ['key_0', 'key_1', 'key_2'])
  pt.columns = ['team', 'x', 'y', 'actions', 'goals']
  pt.goals = pt.goals.astype('int')
  pt['realizations'] = pt.goals / pt.actions

  actions_n = []
  goals_n = []
  for index, row in pt.iterrows():
    team = row.team
    x_gt = row.x - 1
    x_lt = row.x + 1
    y_gt = row.y - 1
    y_lt = row.y + 1
    pt_point = pt[(pt.x <= x_lt) & (pt.x >= x_gt) & (pt.y <= y_lt) & (pt.y >= y_gt) & (pt.team == team)]
    actions_n.append(pt_point.actions.sum())
    goals_n.append(pt_point.goals.sum())

  pt['actions_around'] = actions_n
  pt['goals_around'] = goals_n
  teams = pt.team.unique().tolist()
  print('\nDone')

  return pt, teams

def get_info(team, x, y):
  info_row = pt[(pt.team == team) & (pt.x == x) & (pt.y == y)]
  print(f'Team: {team}\n')
  print(f'Shot position: ({x};{y})\n')
  print(f'Total shots: {int(info_row.actions_around.values)}\n')
  print(f'Total goals: {int(info_row.goals_around.values)}\n')
  print(f'Realization: {round(100*float(info_row.goals_around.values / info_row.actions_around.values),3)}%')

def team_plot(team):
  fig = go.Figure()
  fig.add_trace(go.Scatter(x = pt[pt.team == team].x, 
                          y = pt[pt.team == team].y,
                          mode = 'markers',
                          marker = dict(color = pt[pt.team == team].goals_around / pt[pt.team == team].actions_around, 
                                        colorbar = dict(title = 'Realization'), 
                                        colorscale = 'Inferno'),
                          hovertext = 'Realization: ' + round(100*(pt[pt.team == team].goals_around / pt[pt.team == team].actions_around), 3).astype('str') + '%'
                          ))
  fig.update_xaxes(range=[70, 105])
  fig.update_yaxes(range=[10, 60])
  fig.update_layout(width = 1200, height = 600, title = team)
  fig.show()

pt, teams = preparations()

Loading data... (about 30 seconds)

Done


- Функция **team_plot('название команды')** позволяет получить интерактивный график с информацией об уровне реализации моментов с каждой точки поля (и вокруг неё, согласно условию)
- Функция **get_info('название команды', х, у)** позволяет получить информацию об уровне реализации и количестве моментов с конкретной позиции на поле.

Пример вызова функции team_plot()

In [7]:
team_plot('Juventus')

Все доступные команды хранятся в переменной teams

In [8]:
teams

['Barcelona',
 'CSKA Moskva',
 'Chelsea',
 'Dinamo Moskva',
 'Juventus',
 'Lokomotiv Moskva',
 'Manchester United',
 'Real Madrid',
 'Spartak Moskva',
 'Zenit St. Petersburg']

С любой из них можно вызвать функцию team_plot(). <br>Давайте выведем их все.

In [9]:
for team in teams:
  team_plot(team)

Взглянем также на работу функции get_info(). <br>
Например, для команды Read Madrid, в позиции с координатами (94, 35).

In [10]:
get_info('Real Madrid', 94, 35)

Team: Real Madrid

Shot position: (94;35)

Total shots: 135

Total goals: 64

Realization: 47.407%
